In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class HousingData(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.X.shape[0]

# Instantiate training and test data
train_data = HousingData(X_train, y_train)
test_data = HousingData(X_test, y_test)

# Create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)


In [3]:
from torch import nn
from torch import optim

input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = 1

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(in_features=input_dim, out_features=hidden_dim)
        self.layer_2 = nn.Linear(in_features=hidden_dim, out_features=output_dim)

    def forward(self, x):
        x = self.layer_1(x)
        x = nn.functional.relu(x)
        x = self.layer_2(x)
        return x



model = NeuralNetwork(input_dim, hidden_dim, output_dim)

learning_rate = 0.01
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [4]:
num_epochs = 100
from statistics import mean
loss_values = []

model.train()
for epoch in range(num_epochs):
    for X, y in train_dataloader:
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(-1))
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch+1} | Loss: {mean(loss_values)}")

print("Training Complete")


Epoch: 1 | Loss: 1.2919185351031695
Epoch: 2 | Loss: 0.9568203241210576
Epoch: 3 | Loss: 0.8231542713612857
Epoch: 4 | Loss: 0.7459471826983053
Epoch: 5 | Loss: 0.6954145967382793
Epoch: 6 | Loss: 0.6591054353891879
Epoch: 7 | Loss: 0.6313209860675358
Epoch: 8 | Loss: 0.6095845866948366
Epoch: 9 | Loss: 0.5920760794883654
Epoch: 10 | Loss: 0.5774194837944914
Epoch: 11 | Loss: 0.5648809494247228
Epoch: 12 | Loss: 0.5540601778101628
Epoch: 13 | Loss: 0.5445662426873908
Epoch: 14 | Loss: 0.5360970367055596
Epoch: 15 | Loss: 0.5284805321685719
Epoch: 16 | Loss: 0.5216003885608196
Epoch: 17 | Loss: 0.5153286302546307
Epoch: 18 | Loss: 0.5097640468470245
Epoch: 19 | Loss: 0.5044775192942341
Epoch: 20 | Loss: 0.4995887927017933
Epoch: 21 | Loss: 0.49505831248406423
Epoch: 22 | Loss: 0.4907892578169231
Epoch: 23 | Loss: 0.48680452877397606
Epoch: 24 | Loss: 0.48305671532621525
Epoch: 25 | Loss: 0.4795155458233153
Epoch: 26 | Loss: 0.47625702564683997
Epoch: 27 | Loss: 0.4732666909972758
Epoch:

In [5]:
with torch.no_grad():
    total_loss = 0
    for X, y in test_dataloader:
        outputs = model(X)
        loss = loss_fn(outputs, y.unsqueeze(-1))
        total_loss += loss.item()

    print(f"Test Loss: {total_loss / len(test_dataloader)}")


Test Loss: 0.34630037202284886


Accuracy of the network on the 4128 test instances: 0.2180232558139535%
